In [167]:
import pandas as pd
import numpy as np
import plotly.express as px

In [168]:
X_train = pd.read_csv(r"C:\Users\user\Desktop\daniel\DATA SCIENCE\HENRY\HENRY BOOTCAMP\HACKATON\Hackathon\Housing Dreams\house_train_raw.csv")
X_test = pd.read_csv(r"C:\Users\user\Desktop\daniel\DATA SCIENCE\HENRY\HENRY BOOTCAMP\HACKATON\Hackathon\Housing Dreams\houses_test_raw.csv")


In [169]:
print(X_train.shape,X_test.shape)

(1460, 81) (1459, 80)


In [170]:
X_train.isnull().sum()[X_train.isnull().sum() > 0]
#No hay ningun valor NULO en la columna target "SalePrice", asi que no le hacemos nada mas

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [171]:
cols_with_high_nan = [col for col in X_train.columns if X_train[col].isnull().sum() > 500]
cols_with_high_nan

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

## Imputacion de valores faltantes en los campos con mas valores nulos:

In [172]:
#NA en PoolQC significa que la casa no tiene piscina, eso explica la gran cantidad de valores faltantes:
X_train.loc[X_train["PoolQC"].isnull(), "PoolQC"] = "None"
X_test.loc[X_test["PoolQC"].isnull(), "PoolQC"] = "None"

In [173]:
#Ahora revisaremos el campo MiscFeature
#Revisando encontramos que este campo cuando tiene valores nulos significa que la casa no tiene caracteristicas distintas
X_train.loc[X_train["MiscFeature"].isnull(), "MiscFeature"] = "None"
X_test.loc[X_test["MiscFeature"].isnull(), "MiscFeature"] = "None"

In [174]:
#Ahora revisaremos el campo Alley
#Revisando encontramos que en este campo los valores nulos representan una casa sin acceso por callejon, asi que seteamos el valor "None"
X_train.loc[X_train["Alley"].isnull(), "Alley"] = "None"
X_test.loc[X_test["Alley"].isnull(), "Alley"] = "None"

In [175]:
#Ahora, al investigar la columna Fence, comprobamos que cuando hay valores nulos, significa que la casa no tiene Valla. Asi que arreglamos esto
X_train.loc[X_train["Fence"].isnull(), "Fence"] = "No Fence"
X_test.loc[X_test["Fence"].isnull(), "Fence"] = "No Fence"

In [176]:
#Ahora, al investigar la columna FireplaceQu, comprobamos que cuando hay valores nulos, significa que la casa no tiene Chimenea. Asi que arreglamos esto
X_train.loc[X_train["FireplaceQu"].isnull(), "FireplaceQu"] = "No Fireplace"
X_test.loc[X_test["FireplaceQu"].isnull(), "FireplaceQu"] = "No Fireplace"

#Investigando LotFrontage, no se tiene una razon especifica por la cual falten valores en este campo, asi que idearemos un metodo para imputar sus valores faltantes


In [177]:
#Verificamos los outliers de LotFrontage:
px.box(X_train,y="LotFrontage")

In [178]:
#LotFrontage tiene outliers por lo tanto me deshare de ellos para mi imputacion de valores nulos
q3_, q1_ = np.nanpercentile(np.array(X_train.LotFrontage.values), [75,25])
iqr_ = q3_-q1_
h_outliers_, l_outliers_ = q3_+1.5*iqr_, q1_-1.5*iqr_

In [179]:
fig = px.box(X_train[(X_train.LotFrontage < h_outliers_) & (X_train.LotFrontage > l_outliers_) ],x="Neighborhood",y="LotFrontage")
fig.show()

In [180]:
#Aqui agrupo por neighborhood para sacar la media de LotFrontage y asi crear un diccionario de 
mean_lotfront_per_neighbour = pd.DataFrame(X_train[(X_train.LotFrontage < h_outliers_) & (X_train.LotFrontage > l_outliers_) ].groupby("Neighborhood")["LotFrontage"].mean())
map_dict = {mean_lotfront_per_neighbour.LotFrontage.index[i] : mean_lotfront_per_neighbour.LotFrontage.values[i] for i in range(len(mean_lotfront_per_neighbour.LotFrontage.index))}

In [181]:
X_train.loc[X_train.LotFrontage.isnull(),"LotFrontage"] = X_train[X_train.LotFrontage.isnull()].Neighborhood.map(map_dict)

## Imputacion de las demas variables

In [182]:
X_train.isna().sum()[X_train.isna().sum()>0]

MasVnrType       8
MasVnrArea       8
BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
Electrical       1
GarageType      81
GarageYrBlt     81
GarageFinish    81
GarageQual      81
GarageCond      81
dtype: int64

In [184]:
#Hay 81 valores nulos en las columnas relacionadas con el garage, revisando el diccionario, verificamos que se debe a que estas casas no tienen un garage, asi que imputamos los valores nulos por NONE
for col in ["GarageType","GarageFinish","GarageQual","GarageCond"]:
    X_train.loc[X_train[col].isnull(),col] = "No Garage"
X_train.loc[X_train["GarageYrBlt"].isnull(),"GarageYrBlt"] = 0
#Perfecto, hemos imputado los valores faltantes de estas columnas.

In [185]:
#Se realizara el mismo procedimiento para los campos "Basement"
for col in ["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2"]:
    X_train.loc[X_train[col].isnull(),col] = "No Basement"
X_train.isna().sum()[X_train.isna().sum()>0]

MasVnrType    8
MasVnrArea    8
Electrical    1
dtype: int64

In [186]:
#Para el campo MasVnrType verificamos que los campos con NA es porque no tienen chapa de mamposteria
X_train.loc[X_train["MasVnrType"].isnull(),"MasVnrType"] = "None"
X_train.loc[X_train["MasVnrArea"].isnull(),"MasVnrArea"] = 0
X_train.isna().sum()[X_train.isna().sum()>0]

Electrical    1
dtype: int64

## Codificacion de Variables categoricas 

In [187]:
#Hice una exploracion de cada campo, verificando si es una variable ordinal o nominal, ya que hay diferencia entre este tipo de variables categoricas
#cada una se codifica de una manera distinta. Las variables que pueden ser ordenadas las almacene en "ordinal_cols"
from sklearn.preprocessing import LabelEncoder
ordinal_cols = ["Street", "LotShape", "LandContour", "Utilities", "LandSlope", "MSSubClass", "OverallQual","OverallCond","ExterQual","ExterCond","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","HeatingQC","CentralAir","Electrical","KitchenQual","FireplaceQu","GarageFinish","GarageQual","GarageCond","PoolQC","Fence"]
#Ahora se transforman:
for col in ordinal_cols:
    le = LabelEncoder()
    le.fit(list(X_train[col].values)) 
    X_train[col] = le.transform(list(X_train[col].values))



In [188]:
#Variables categoricas nominales:
categorical_variables = [col for col in X_train.columns if X_train[col].dtype == "object"]
len(categorical_variables)

21

In [189]:
#Importamos lo necesario para realizar el One Hot encoder:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([('ohe', OneHotEncoder(handle_unknown="ignore",sparse=False), categorical_variables)],#Los nuevos valores que se encuentren en las columnas de test, los ignorará
                         remainder='passthrough') #transformamos las columnas categoricas, y las demas cols las pasamos de largo
                         
OH_X_train = pd.DataFrame(ct.fit_transform(X_train), columns=ct.get_feature_names_out())
#OH_X_valid = pd.DataFrame(ct.transform(X_test), columns=ct.get_feature_names_out())

OH_X_train.index = X_train.index 
#OH_X_valid.index = X_test.index

In [191]:
#Aqui dropeamos los valores nulos de las demas columnas "Electrical" solo 1 valor nulo
X_train.dropna(axis=0,inplace=True)
#X_test.dropna(axis=0,inplace=True)
y_train = X_train.pop("SalePrice")
print(X_train.shape,OH_X_train.shape)

KeyError: 'SalePrice'

In [192]:
print(X_train.shape,OH_X_train.shape)

(1460, 80) (1460, 226)


## Creacion del modelo y obtencion del error


In [193]:
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor

bosque = RandomForestRegressor(n_estimators=200, random_state=0) #Instanciamos el modelo
bosque.fit(OH_X_train, y_train) #Fiteamos el modelo con nuestra data de train
y_pred = bosque.predict(OH_X_train) #Predecimos para esa misma data


print(mean_squared_log_error(y_train,y_pred,squared=False))

0.004222084149683005
